### Change Log
---
handelling grid search params and correcting SAL module

In [1]:
import sys
sys.path.append('/data/extensibleFramework/')

In [2]:
from extensibleFramework.src.models import convolution_neural_network
from extensibleFramework.src.models import recurrent_nn_with_attention
from extensibleFramework.src.models import feed_forward_network
from extensibleFramework.src.models import extra_layers
from extensibleFramework.src.utils import saving_and_loading
from extensibleFramework.src.utils import grid_search
from extensibleFramework.src.models import config
import torch
import numpy as np
import json

In [3]:
import pandas as pd
import chakin
import matplotlib.pyplot as plt
from torchtext import data
import nltk
import json
from torchtext import vocab
from tqdm import tqdm
import torch
import random
import re
import pandas as pd
import os
import numpy as np
import sys
import random
import tarfile
import urllib
from torchtext import data
import datetime
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
data_frame = pd.read_csv("../data/crude/labeledTrainData.tsv",sep="\t")
data_frame = data_frame[['review', 'sentiment']]
data_block  = data_frame.values

In [5]:
SEED = 1
split = 0.80
batch_size  =32

In [6]:
# random.shuffle(data_block)

# train_file = open('../data/processed/train.json', 'w')
# test_file = open('../data/processed/test.json', 'w')
# for i in  range(0,int(len(data_block)*split)):
#     train_file.write(str(json.dumps({'review' : data_block[i][0], 'label' : data_block[i][1]}))+"\n")
# for i in  range(int(len(data_block)*split),len(data_block)):
#     test_file.write(str(json.dumps({'review' : data_block[i][0], 'label' : data_block[i][1]}))+"\n")

In [7]:
def tokenize(sentiments):
    
    return nltk.tokenize.word_tokenize(sentiments)
# def pad_to_equal(x):
#     if len(x) < 61:
#         return x + ['<pad>' for i in range(0, 61 - len(x))]
#     else:
#         return x[:61]
def to_categorical(x):
    x = int(x)
    if x == 1:
        return [0,1]
    if x == 0:
        return [1,0]

In [8]:
REVIEW = data.Field(sequential=True , tokenize=tokenize, use_vocab = True, lower=True,batch_first=True)
LABEL = data.Field(is_target=True,use_vocab = False, sequential=False, preprocessing = to_categorical)
fields = {'review': ('review', REVIEW), 'label': ('label', LABEL)}
train_data , test_data = data.TabularDataset.splits(
                            path = '',
                            train = '../data/processed/train.json',
                            test = '../data/processed/test.json',
                            format = 'json',
                            fields = fields)


In [9]:
# chakin.search(lang='English')

In [10]:
# chakin.download(number=12, save_dir = "./")

In [11]:
# !unzip glove.6B.zip

In [12]:
vec = vocab.Vectors(name = "../embedidngs/glove.6B.100d.txt",cache = "./")

In [13]:
REVIEW.build_vocab(train_data, test_data, max_size=400000, vectors=vec)

In [14]:
sentiment_vocab = REVIEW.vocab

In [15]:
len(sentiment_vocab)

64171

In [16]:
class parameters:
    def __init__(self):
        self.cnn_rnn_vocab_size = len(sentiment_vocab)
        self.cnn_rnn_embed_dim = 100
        self.cnn_rnn_class_num = [50,100,200]
        self.cnn_out_channel_num = [8,12]
        self.cnn_kernel_sizes =  [3,4,5]
        self.rnn_n_layers = [1,2,3]
        self.rnn_hidden_size =  [128, 256]
        self.use_pretrained_weights = True
        self.cnn_rnn_weights =  sentiment_vocab.vectors
        self.cnn_rnn_weight_is_trainable =   False
        self.dropout = [0.2, 0.5]
        self.batch_size = batch_size
        self.merge_mode = "CONCAT"
        self.ffn_activation =  "Relu"
        self.ffn_final_output_classes =  2
        self.ffn_perceptron_per_layer = [[50, 25], [100,50, 25],  [200,100,50, 25]]
        self.ffn_layer_wise_dropout = 0.2
        self.learning_rate =  [0.1,0.2]
        self.momentum = [0.5,0.9]
        self.device = device
    

In [17]:
SP = grid_search.searchParameters(parameters)


In [18]:
train_iter, test_iter = data.Iterator.splits(
            (train_data, test_data), sort_key=lambda x: len(x.review),
            batch_sizes=(batch_size,batch_size), device=device)

In [19]:
for batch in test_iter:
    feature, target = batch.review, batch.label
    print(feature.data.shape, target.data.shape)
    break

torch.Size([32, 47]) torch.Size([32, 2])


In [20]:
sentiment_vocab.vectors.shape

torch.Size([64171, 100])

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class ensemble_model(nn.Module):
    def __init__(self, config_object):
        super(ensemble_model, self).__init__() 
        self.cnn = convolution_neural_network.cnn_text(config_object)
        self.rnn_attention = recurrent_nn_with_attention.RNNAttentionModel(config_object)
        self.merge_layer = extra_layers.MergeAndFlattern(config_object)
        self.ffn = feed_forward_network.ffn(config_object)
    def forward(self, x):
        cnn_output = self.cnn(x)
        rnnAttention_output = self.rnn_attention(x)
        merge_layer_output = self.merge_layer(cnn_output,rnnAttention_output )
        final_output = self.ffn(merge_layer_output)
        
#         print("cnn_output : ",cnn_output.shape)
#         print("rnn_input : ",rnnAttention_output.shape)
#         print("merge_layer_output : ", merge_layer_output.shape)
#         print("final_output : ", final_output.shape)
        return final_output

In [22]:
for parma_set in SP.random_search(1):
    configuration = config.Config(parma_set)


In [38]:
for parma_set in SP.random_search(1):
    configuration = config.Config(parma_set)
    EM  = ensemble_model(configuration)
    EM  = EM.to(device)
#     print(json.dumps(configuration, default=lambda x: x.__dict__))


64171 100 200 12 [3, 4, 5] 0.2 100


In [24]:
cnn_input = torch.Tensor(np.random.randint(5, size=[8, 10])).long().to(device)
rnn_input = torch.Tensor(np.random.random([8, 10])).long().to(device) #batch_size, input_size

In [25]:
# EM(rnn_input)

In [26]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    rounded_preds = torch.argmax(preds, dim=1)
    correct = (rounded_preds == torch.argmax(y, dim=1)).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [27]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        feature, target = batch.review, batch.label
        optimizer.zero_grad()
        predictions = model(feature.to(device))            
        loss = criterion(predictions.type(torch.FloatTensor), target.type(torch.FloatTensor))
        loss.backward()
        optimizer.step()
        acc = binary_accuracy(predictions.type(torch.FloatTensor), target.type(torch.FloatTensor))
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return model, epoch_loss / len(iterator), epoch_acc / len(iterator)

In [28]:
def test_accuracy_calculator(model, test_iterator):
    epoch_acc = 0
    for batch in test_iterator:
        if batch.review.shape[0] ==  32:
            feature, target = batch.review, batch.label
            predictions = model(feature.to(device))            
            acc = binary_accuracy(predictions.type(torch.FloatTensor), target.type(torch.FloatTensor))
            epoch_acc += acc.item()
    return  epoch_acc / len(test_iterator)

In [29]:
optimizer = torch.optim.SGD(EM.parameters(), lr=0.1,momentum=0.9)
criterion = nn.MSELoss()
criterion = criterion.to(device)

In [30]:
epochs  = 100
log_interval = 1
loss = []
accuracy = []
test_accuracy = []
for i in tqdm(range(epochs)):
    if (i != 0 and i%10 == 0 ):
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr']/2
        print (" === New Learning rate : ", param_group['lr'], " === ")
    
    model, epoch_loss, epoch_acc = train(EM, train_iter, optimizer, criterion)
    
    test_acc = test_accuracy_calculator(model, test_iter)
    accuracy.append(epoch_acc)
    loss.append(epoch_loss)
    test_accuracy.append(test_acc)
    print(epoch_acc,test_acc,epoch_loss)

In [31]:
SAL = saving_and_loading.objectManager()

In [32]:
SAL.saver(EM, "train")

>> train/20190306-144431 train
++++++++ Model dumped with following parameters ++++++++
--------------------------------------------------------
cnn.embed.weight 	 torch.Size([64171, 100])
cnn.convs1.0.weight 	 torch.Size([8, 1, 3, 3])
cnn.convs1.0.bias 	 torch.Size([8])
cnn.convs1.1.weight 	 torch.Size([8, 1, 4, 4])
cnn.convs1.1.bias 	 torch.Size([8])
cnn.convs1.2.weight 	 torch.Size([8, 1, 5, 5])
cnn.convs1.2.bias 	 torch.Size([8])
cnn.fc1.weight 	 torch.Size([100, 24])
cnn.fc1.bias 	 torch.Size([100])
rnn_attention.word_embeddings.weight 	 torch.Size([64171, 100])
rnn_attention.lstm.weight_ih_l0 	 torch.Size([1024, 100])
rnn_attention.lstm.weight_hh_l0 	 torch.Size([1024, 256])
rnn_attention.lstm.bias_ih_l0 	 torch.Size([1024])
rnn_attention.lstm.bias_hh_l0 	 torch.Size([1024])
rnn_attention.label.weight 	 torch.Size([100, 256])
rnn_attention.label.bias 	 torch.Size([100])
ffn.network.input.weight 	 torch.Size([50, 200])
ffn.network.input.bias 	 torch.Size([50])
ffn.network.Linear0.

In [34]:
EM = SAL.loader("train/20190306-144431/ensemble_model.ckpt")

In [36]:
EM.state_dict

AttributeError: 'collections.OrderedDict' object has no attribute 'state_dict'